In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from ddml import *

In [96]:
import glob
from utils.ihdp_data import *


data_base_dir = "./data/IHDP/sim_data_redraw_T"
simulation_files = sorted(glob.glob("{}/*.csv".format(data_base_dir)))
  
# Simulation parameters
exp_name = "IHDP"
np.random.seed(0)

data_base_dir = "./data/IHDP/sim_data_redraw_T"
simulation_files = sorted(glob.glob("{}/*.csv".format(data_base_dir)))

# Generate covariates
np.random.seed(0)

true_ATEs_list = []

nn_ls_list = []
nn_ls_logit_list = []
nn_kl_list = []
nn_tl_list = []
nn_mle_list = []

rkhs_ls_list = []
rkhs_kl_list = []
rkhs_tl_list = []
rkhs_mle_list = []

result_list = []

for tr in range(len(simulation_files)):
    print("tr", tr)
    simulation_file = simulation_files[tr]
    x = load_and_format_covariates(simulation_file, delimiter=' ')
    t, y, y_cf, mu_0, mu_1 = load_other_stuff(simulation_file, delimiter=' ')
    true_ATE = np.mean(mu_1 - mu_0)
    true_ATEs_list.append(true_ATE)
    
    covariates = x
    treatment = t.T[0]
    outcome = y.T[0]
    
    # Below, we conduct each method to estimate the ATE

    ############################

    cross_fitting_folds=2

    method = "NN_RieszLearner"

    riesz_hidden_dim= 200
    reg_hidden_dim= 100
    riesz_max_iter = 200
    reg_max_iter = 100
    
    riesz_loss = "LS"
    riesz_with_D = True
    riesz_link_name = "Linear"

    dd_ate = DDML_ATE()
    result = dd_ate.estimate(covariates, treatment, 
                    outcome, method=method, 
                    cross_fitting_folds = cross_fitting_folds,
                    riesz_loss=riesz_loss, 
                    riesz_with_D=riesz_with_D, 
                    riesz_link_name=riesz_link_name, 
                    riesz_hidden_dim=riesz_hidden_dim, 
                    reg_hidden_dim=reg_hidden_dim,
                    riesz_max_iter=riesz_max_iter, 
                    reg_max_iter=reg_max_iter)

    nn_ls_list.append(result)
    

    riesz_loss = "KL"
    riesz_with_D = False
    riesz_link_name = "Logit"

    dd_ate = DDML_ATE()
    result = dd_ate.estimate(covariates, treatment, 
                    outcome, method=method, 
                    cross_fitting_folds = cross_fitting_folds,
                    riesz_loss=riesz_loss, 
                    riesz_with_D=riesz_with_D, 
                    riesz_link_name=riesz_link_name, 
                    riesz_hidden_dim=riesz_hidden_dim, 
                    reg_hidden_dim=reg_hidden_dim,
                    riesz_max_iter=riesz_max_iter, 
                    reg_max_iter=reg_max_iter)

    nn_kl_list.append(result)

    riesz_loss = "TL"
    riesz_with_D = False
    riesz_link_name = "Logit"

    dd_ate = DDML_ATE()
    result = dd_ate.estimate(covariates, treatment, 
                    outcome, method=method, 
                    cross_fitting_folds = cross_fitting_folds,
                    riesz_loss=riesz_loss, 
                    riesz_with_D=riesz_with_D, 
                    riesz_link_name=riesz_link_name, 
                    riesz_hidden_dim=riesz_hidden_dim, 
                    reg_hidden_dim=reg_hidden_dim,
                    riesz_max_iter=riesz_max_iter, 
                    reg_max_iter=reg_max_iter)

    nn_tl_list.append(result)
    
    riesz_loss = "MLE"
    riesz_with_D = False
    riesz_link_name = "Logit"

    dd_ate = DDML_ATE()
    result = dd_ate.estimate(covariates, treatment, 
                    outcome, method=method, 
                    cross_fitting_folds = cross_fitting_folds,
                    riesz_loss=riesz_loss, 
                    riesz_with_D=riesz_with_D, 
                    riesz_link_name=riesz_link_name, 
                    riesz_hidden_dim=riesz_hidden_dim, 
                    reg_hidden_dim=reg_hidden_dim,
                    riesz_max_iter=riesz_max_iter, 
                    reg_max_iter=reg_max_iter)

    nn_mle_list.append(result)

    ##############################

    method = "RKHS_RieszLearner"

    folds = 3
    num_basis = 100


    riesz_loss = "LS"
    riesz_with_D = True
    riesz_link_name = "Linear"
    is_separate = False

    result = dd_ate.estimate(covariates, treatment, outcome, method=method, cross_fitting_folds = cross_fitting_folds, riesz_loss=riesz_loss, riesz_with_D=riesz_with_D, riesz_link_name=riesz_link_name, is_separate=is_separate, folds=folds, num_basis=num_basis)
    rkhs_ls_list.append(result)


    riesz_loss = "KL"
    riesz_with_D = False
    riesz_link_name = "Logit"
    is_separate = False

    result = dd_ate.estimate(covariates, treatment, outcome, method=method, cross_fitting_folds = cross_fitting_folds, riesz_loss=riesz_loss, riesz_with_D=riesz_with_D, riesz_link_name=riesz_link_name, is_separate=is_separate, folds=folds, num_basis=num_basis)
    rkhs_kl_list.append(result)


    riesz_loss = "TL"
    riesz_with_D = False
    riesz_link_name = "Logit"
    is_separate = False

    result = dd_ate.estimate(covariates, treatment, outcome, method=method, cross_fitting_folds = cross_fitting_folds, riesz_loss=riesz_loss, riesz_with_D=riesz_with_D, riesz_link_name=riesz_link_name, is_separate=is_separate, folds=folds, num_basis=num_basis)
    rkhs_tl_list.append(result)

    riesz_loss = "MLE"
    riesz_with_D = False
    riesz_link_name = "Logit"
    is_separate = False

    result = dd_ate.estimate(covariates, treatment, outcome, method=method, cross_fitting_folds = cross_fitting_folds, riesz_loss=riesz_loss, riesz_with_D=riesz_with_D, riesz_link_name=riesz_link_name, is_separate=is_separate, folds=folds, num_basis=num_basis)
    rkhs_mle_list.append(result)
    
    np.savetxt("results3/true_ATEs_%s.csv"%exp_name, true_ATEs_list)
    np.savetxt("results3/nn_ls_%s.csv"%exp_name, nn_ls_list)
    np.savetxt("results3/nn_kl_%s.csv"%exp_name, nn_kl_list)
    np.savetxt("results3/nn_tl_%s.csv"%exp_name, nn_tl_list)
    np.savetxt("results3/nn_mle_%s.csv"%exp_name, nn_mle_list)
    np.savetxt("results3/rkhs_ls_%s.csv"%exp_name, rkhs_ls_list)
    np.savetxt("results3/rkhs_kl_%s.csv"%exp_name, rkhs_kl_list)
    np.savetxt("results3/rkhs_tl_%s.csv"%exp_name, rkhs_tl_list)
    np.savetxt("results3/rkhs_mle_%s.csv"%exp_name, rkhs_mle_list)

tr 0
tr 1
tr 2
tr 3
tr 4
tr 5
tr 6
tr 7
tr 8
tr 9
tr 10
tr 11
tr 12
tr 13
tr 14
tr 15
tr 16
tr 17
tr 18
tr 19
tr 20


/Users/mkato/Dropbox/DirectDebiasedMachineLearning/rkhs_rieszlearner.py:200: RuntimeWarning: divide by zero encountered in log
  - treatment * np.log(-alpha0 - 1)
/Users/mkato/Dropbox/DirectDebiasedMachineLearning/rkhs_rieszlearner.py:200: RuntimeWarning: invalid value encountered in multiply
  - treatment * np.log(-alpha0 - 1)
/Users/mkato/Dropbox/DirectDebiasedMachineLearning/rkhs_rieszlearner.py:128: RuntimeWarning: invalid value encountered in divide
  alpha = treatment / ex1 - (1.0 - treatment) / (1.0 - ex0)
/Users/mkato/Dropbox/DirectDebiasedMachineLearning/rkhs_rieszlearner.py:128: RuntimeWarning: divide by zero encountered in divide
  alpha = treatment / ex1 - (1.0 - treatment) / (1.0 - ex0)
/Users/mkato/Dropbox/DirectDebiasedMachineLearning/rkhs_rieszlearner.py:202: RuntimeWarning: invalid value encountered in multiply
  - (1 - treatment) * alpha0
/Users/mkato/Dropbox/DirectDebiasedMachineLearning/rkhs_rieszlearner.py:126: RuntimeWarning: overflow encountered in exp
  ex1 = 1.

tr 21
tr 22
tr 23
tr 24
tr 25
tr 26
tr 27
tr 28
tr 29
tr 30
tr 31
tr 32
tr 33
tr 34
tr 35
tr 36
tr 37
tr 38
tr 39
tr 40
tr 41
tr 42
tr 43
tr 44
tr 45
tr 46
tr 47
tr 48
tr 49
tr 50
tr 51
tr 52
tr 53
tr 54
tr 55
tr 56
tr 57
tr 58
tr 59
tr 60
tr 61
tr 62
tr 63
tr 64
tr 65
tr 66
tr 67
tr 68
tr 69
tr 70
tr 71
tr 72
tr 73
tr 74
tr 75
tr 76
tr 77
tr 78
tr 79
tr 80
tr 81
tr 82
tr 83
tr 84
tr 85
tr 86
tr 87
tr 88
tr 89
tr 90
tr 91
tr 92
tr 93
tr 94
tr 95
tr 96
tr 97
tr 98
tr 99


In [104]:
import pandas as pd

In [89]:
nn_kl_list

[(np.float32(4.0798154),
  np.float64(7.0919091415054725),
  np.float64(3.915009803176155),
  np.float32(4.0669913),
  np.float32(4.0926394),
  np.float64(4.174764950220169),
  np.float64(10.009053332790776),
  np.float64(3.3530192299880497),
  np.float64(4.47700037636426)),
 (np.float32(3.1337237),
  np.float64(9.863595513032811),
  np.float64(3.6516029914026062),
  np.float32(3.0788546),
  np.float32(3.188593),
  np.float64(-2.511092630313618),
  np.float64(22.238283656379238),
  np.float64(-0.12967721855660486),
  np.float64(7.432883201361817))]

In [90]:
nn_ls_list

[(np.float32(4.0910077),
  np.float64(8.791248008878828),
  np.float64(4.108682303430705),
  np.float32(4.0786266),
  np.float32(4.103389),
  np.float64(5.484835507627786),
  np.float64(12.09766051012987),
  np.float64(3.5629847138098008),
  np.float64(4.65437989305161)),
 (np.float32(3.1990237),
  np.float64(9.248385019136222),
  np.float64(0.9748679962379028),
  np.float32(3.1427956),
  np.float32(3.255252),
  np.float64(-4.980711413180753),
  np.float64(23.477481451453198),
  np.float64(-3.3512682823627387),
  np.float64(5.301004274838544))]

In [69]:
true_ATEs_list2 = np.loadtxt("results2/true_ATEs_IHDP.csv")

In [97]:
true_ATEs_list = np.array(true_ATEs_list)

nn_ls_list = np.array(nn_ls_list)
nn_kl_list = np.array(nn_kl_list)
nn_tl_list = np.array(nn_tl_list)
nn_mle_list = np.array(nn_mle_list)

rkhs_ls_list = np.array(rkhs_ls_list)
rkhs_kl_list = np.array(rkhs_kl_list)
rkhs_tl_list = np.array(rkhs_tl_list)
rkhs_mle_list = np.array(rkhs_mle_list)

In [110]:
np.max(nn_ls_list)

np.float64(52.15854595030291)

In [117]:
results = np.array([
    np.nanmean(
        np.where(
            (nn_ls_list[:, :3]) ** 2 <= 60,
            (nn_ls_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (nn_kl_list[:, :3]) ** 2 <= 60,
            (nn_kl_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (nn_tl_list[:, :3]) ** 2 <= 60,
            (nn_tl_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (nn_mle_list[:, :3]) ** 2 <= 60,
            (nn_mle_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (rkhs_ls_list[:, :3]) ** 2 <= 60,
            (rkhs_ls_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (rkhs_kl_list[:, :3]) ** 2 <= 60,
            (rkhs_kl_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (rkhs_tl_list[:, :3]) ** 2 <= 60,
            (rkhs_tl_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (rkhs_mle_list[:, :3]) ** 2 <= 60,
            (rkhs_mle_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    )
])






In [119]:
pd.DataFrame(results).round(3)

,0,1,2
0,1.521,6.816,0.307
1,1.572,9.418,0.439
2,1.554,2.836,0.315
3,1.580,3.000,0.435
4,19.975,3.558,19.972
5,3.502,1.911,4.582
6,2.588,1.776,4.448
7,2.481,1.220,2.315


In [74]:
nn_ls_conf_list = [np.mean((nn_ls_list[:, 3] <= true_ATEs_list2) * (true_ATEs_list2 <= nn_ls_list[:, 4])), np.mean((nn_ls_list[:, 5] <= true_ATEs_list2) * (true_ATEs_list2 <= nn_ls_list[:, 6])), np.mean((nn_ls_list[:, 7] <= true_ATEs_list2) * (true_ATEs_list2 <= nn_ls_list[:, 8]))]
nn_ls_conf_list = [np.mean((nn_ls_list[:, 3] <= true_ATEs_list2) * (true_ATEs_list2 <= nn_ls_list[:, 4])), np.mean((nn_ls_list[:, 5] <= true_ATEs_list2) * (true_ATEs_list2 <= nn_ls_list[:, 6])), np.mean((nn_ls_list[:, 7] <= true_ATEs_list2) * (true_ATEs_list2 <= nn_ls_list[:, 8]))]


In [78]:
nn_ls_list[:, 3]

array([1.75428188, 0.09685349, 1.39358461, 0.36798656, 1.20057082,
       0.11121023, 1.6261797 , 0.36892954, 2.01902986, 0.54510945,
       1.80321395, 1.34078765, 2.01722288, 0.18898365, 0.46001995,
       1.47752845, 0.30765808, 0.54882407, 0.41599172, 0.93810433,
       1.41291249, 0.02154516, 0.1565046 , 1.62321317, 0.95216608,
       0.53496999, 1.94705737, 0.50399709, 0.3442356 , 0.5254792 ,
       1.60863709, 0.9682678 , 0.67829937, 0.69943434, 1.96460938,
       0.8413716 , 0.95203239, 1.37265491, 1.58692968, 1.09022319,
       0.51941812, 0.76278985, 1.62482774, 1.93680668, 1.34098458,
       0.57430494, 0.70429301, 1.74443734, 1.63604653, 1.10249865,
       0.41582265, 1.03364038, 1.88992143, 1.98194349, 1.61698377,
       1.24919105, 1.59811759, 1.2432884 , 1.80015945, 1.13445973,
       0.57422632, 1.8700639 , 0.70921475, 0.94404948, 1.57322085,
       1.0375247 , 1.44715726, 1.7006017 , 2.05276561, 0.21680091,
       1.84891868, 1.9963727 , 1.3643173 , 1.99079466, 1.58188

In [77]:
true_ATEs_list2

array([4.19061929, 2.21867106, 4.36578046, 5.15550452, 4.23515674,
       3.25911059, 3.89980253, 3.50141106, 3.95803369, 3.53677565,
       3.91925338, 3.95697207, 4.0492634 , 2.88052387, 4.46885457,
       4.15143145, 6.62991237, 4.11355989, 5.54867359, 4.93824037,
       4.1093892 , 2.37113283, 3.03489245, 4.18631512, 3.27394799,
       2.95040143, 3.92152335, 3.03656141, 3.97523688, 6.37835307,
       4.48636285, 3.9077722 , 4.09318224, 5.5913192 , 4.05232508,
       4.73073735, 4.75426856, 4.00815232, 4.22686211, 4.17225219,
       7.87664592, 4.74549789, 4.18124934, 4.1508128 , 4.46677005,
       8.18767045, 4.06630752, 4.14114241, 3.97812599, 4.58832728,
       4.17367242, 4.51693573, 4.11473611, 4.06517846, 4.22027123,
       4.63698329, 3.84368709, 4.18401259, 4.16924409, 3.92231572,
       3.45770887, 3.9244933 , 3.89238764, 4.94429353, 4.49342595,
       3.93521053, 4.12558656, 3.94613535, 4.03249611, 4.24693152,
       4.33929104, 3.98203664, 4.26268879, 4.01412655, 4.11420

In [65]:
nn_ls_list[:, 4]

array([1.80746806, 0.20685908, 1.47180307, 0.4955194 , 1.28728008,
       0.24348256, 1.68393326, 0.49156746, 2.05771422, 0.66295117,
       1.86068523, 1.42167926, 2.06058598, 0.32678303, 0.57843471,
       1.55034482, 0.43116552, 0.6670028 , 0.56208545, 1.0391376 ,
       1.47921908, 0.14477962, 0.28292572, 1.68613923, 1.05080342,
       0.62983197, 1.98216927, 0.60155129, 0.46549386, 0.65199256,
       1.67335939, 1.05992329, 0.7824071 , 0.81667763, 1.99924922,
       0.95640904, 1.0532198 , 1.44614506, 1.64789236, 1.1943537 ,
       0.65549946, 0.87468648, 1.68428314, 1.97997928, 1.43012142,
       0.71064246, 0.81959939, 1.79044116, 1.70463908, 1.19558108,
       0.53548515, 1.13122582, 1.9256227 , 2.01832414, 1.67562687,
       1.32438922, 1.66281414, 1.32319081, 1.84563994, 1.21533585,
       0.69797391, 1.9197613 , 0.80884033, 1.0431608 , 1.64576447,
       1.12836981, 1.51432121, 1.75769961, 2.08881688, 0.35834855,
       1.88534069, 2.02993059, 1.46270454, 2.03049946, 1.63382

In [66]:
true_ATEs_list

array([4.19061929, 2.21867106, 4.36578046, 5.15550452, 4.23515674,
       3.25911059, 3.89980253, 3.50141106, 3.95803369, 3.53677565,
       3.91925338, 3.95697207, 4.0492634 , 2.88052387, 4.46885457,
       4.15143145, 6.62991237, 4.11355989, 5.54867359, 4.93824037,
       4.1093892 , 2.37113283, 3.03489245, 4.18631512, 3.27394799,
       2.95040143, 3.92152335, 3.03656141, 3.97523688, 6.37835307,
       4.48636285, 3.9077722 , 4.09318224, 5.5913192 , 4.05232508,
       4.73073735, 4.75426856, 4.00815232, 4.22686211, 4.17225219,
       7.87664592, 4.74549789, 4.18124934, 4.1508128 , 4.46677005,
       8.18767045, 4.06630752, 4.14114241, 3.97812599, 4.58832728,
       4.17367242, 4.51693573, 4.11473611, 4.06517846, 4.22027123,
       4.63698329, 3.84368709, 4.18401259, 4.16924409, 3.92231572,
       3.45770887, 3.9244933 , 3.89238764, 4.94429353, 4.49342595,
       3.93521053, 4.12558656, 3.94613535, 4.03249611, 4.24693152,
       4.33929104, 3.98203664, 4.26268879, 4.01412655, 4.11420

In [63]:
nn_ls_list[:, 3] <= true_ATEs_list

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [62]:
nn_ls_conf_list

[np.float64(0.0), np.float64(0.0), np.float64(0.0)]

In [121]:
nn_ls_conf_list = [np.mean((nn_ls_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= nn_ls_list[:, 4])), np.mean((nn_ls_list[:, 5] <= true_ATEs_list) * (5 <= nn_ls_list[:, 6])), np.mean((nn_ls_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= nn_ls_list[:, 8]))]
nn_kl_conf_list = [np.mean((nn_kl_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= nn_kl_list[:, 4])), np.mean((nn_kl_list[:, 5] <= true_ATEs_list) * (5 <= nn_kl_list[:, 6])), np.mean((nn_kl_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= nn_kl_list[:, 8]))]
nn_tl_conf_list = [np.mean((nn_tl_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= nn_tl_list[:, 4])), np.mean((nn_tl_list[:, 5] <= true_ATEs_list) * (5 <= nn_tl_list[:, 6])), np.mean((nn_tl_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= nn_tl_list[:, 8]))]
nn_mle_conf_list = [np.mean((nn_mle_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= nn_mle_list[:, 4])), np.mean((nn_mle_list[:, 5] <= true_ATEs_list) * (5 <= nn_mle_list[:, 6])), np.mean((nn_mle_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= nn_mle_list[:, 8]))]
rkhs_ls_conf_list = [np.mean((rkhs_ls_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= rkhs_ls_list[:, 4])), np.mean((rkhs_ls_list[:, 5] <= true_ATEs_list) * (5 <= rkhs_ls_list[:, 6])), np.mean((rkhs_ls_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= rkhs_ls_list[:, 8]))]
rkhs_kl_conf_list = [np.mean((rkhs_kl_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= rkhs_kl_list[:, 4])), np.mean((rkhs_kl_list[:, 5] <= true_ATEs_list) * (5 <= rkhs_kl_list[:, 6])), np.mean((rkhs_kl_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= rkhs_kl_list[:, 8]))]
rkhs_tl_conf_list = [np.mean((rkhs_tl_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= rkhs_tl_list[:, 4])), np.mean((rkhs_tl_list[:, 5] <= true_ATEs_list) * (5 <= rkhs_tl_list[:, 6])), np.mean((rkhs_tl_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= rkhs_tl_list[:, 8]))]
rkhs_mle_conf_list = [np.mean((rkhs_mle_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= rkhs_mle_list[:, 4])), np.mean((rkhs_mle_list[:, 5] <= true_ATEs_list) * (5 <= rkhs_mle_list[:, 6])), np.mean((rkhs_mle_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= rkhs_mle_list[:, 8]))]


In [122]:
results2 = np.array([
                  nn_ls_conf_list,
                  nn_kl_conf_list,
                  nn_tl_conf_list,
                  nn_mle_conf_list,
                  rkhs_ls_conf_list,
                  rkhs_kl_conf_list,
                  rkhs_tl_conf_list,
                  rkhs_mle_conf_list])

In [123]:
results2

array([[0.03, 0.41, 1.  ],
       [0.06, 0.08, 1.  ],
       [0.03, 0.73, 0.94],
       [0.01, 0.61, 0.9 ],
       [0.  , 0.  , 0.  ],
       [0.34, 0.91, 0.82],
       [0.48, 0.93, 0.88],
       [0.39, 0.81, 0.84]])

In [50]:
mse_kl = (nn_kl_list[:, :3] - np.array([true_ATEs_list]*3).T)**2

In [126]:
print(pd.DataFrame([results.flatten(), results2.flatten()]).to_latex(float_format="%.2f"))

\begin{tabular}{lrrrrrrrrrrrrrrrrrrrrrrrr}
\toprule
 & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 & 20 & 21 & 22 & 23 \\
\midrule
0 & 1.52 & 6.82 & 0.31 & 1.57 & 9.42 & 0.44 & 1.55 & 2.84 & 0.32 & 1.58 & 3.00 & 0.43 & 19.98 & 3.56 & 19.97 & 3.50 & 1.91 & 4.58 & 2.59 & 1.78 & 4.45 & 2.48 & 1.22 & 2.32 \\
1 & 0.03 & 0.41 & 1.00 & 0.06 & 0.08 & 1.00 & 0.03 & 0.73 & 0.94 & 0.01 & 0.61 & 0.90 & 0.00 & 0.00 & 0.00 & 0.34 & 0.91 & 0.82 & 0.48 & 0.93 & 0.88 & 0.39 & 0.81 & 0.84 \\
\bottomrule
\end{tabular}

